**ЗАДАЧА ПРОЕКТА:**  
- изучить событийную аналитику пользователей на сайте - воронку продаж, узнайть, как и сколько пользователей доходят до покупки,  
а сколько — «застревает» на предыдущих шагах. На каких именно?
- исследовать результат А/А/В-эксперимента и выяснить какой шрифт лучше

### Данные

Выгрузим датасет и ознакомимся с информацией

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from datetime import date
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import scipy.stats as stats
from scipy import stats as st
import math as mth
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

In [2]:
import warnings
warnings.filterwarnings("ignore")

Сразу переименуем столбцы в удобные для нас:

In [3]:
try:
    data = pd.read_csv('/datasets/logs_exp.csv', sep="\t")
except:
    data = pd.read_csv('https://code.s3.yandex.net//datasets/logs_exp.csv', sep="\t")
    
data = data.rename(columns={'EventName' : 'event_name', 'DeviceIDHash' : 'id', 'EventTimestamp' : 'event_timestamp', 'ExpId' : 'exp_id'})    
display(data)

,event_name,id,event_timestamp,exp_id
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248
...,...,...,...,...
244121,MainScreenAppear,4599628364049201812,1565212345,247
244122,MainScreenAppear,5849806612437486590,1565212439,246
244123,MainScreenAppear,5746969938801999050,1565212483,246
244124,MainScreenAppear,5746969938801999050,1565212498,246


**Описание данных**  
Каждая запись в логе — это действие пользователя, или событие:
- event_name — название события;
- id — уникальный идентификатор пользователя;
- event_timestamp	 — время события;
- exp_id — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.

### Подготовка данных

Проверим пропуски в данных и дубликаты:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_name       244126 non-null  object
 1   id               244126 non-null  int64 
 2   event_timestamp  244126 non-null  int64 
 3   exp_id           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


Пропусков нет.

Переведем столбец даты и время в нужный формат:

In [5]:
data['event_timestamp'] = pd.to_datetime(data['event_timestamp'], unit='s')

Проверим явные дубликаты и удалим их, если есть

In [6]:
data.duplicated().sum()

413

In [7]:
data = data.drop_duplicates()

Создадим отдельный столбец даты для удобства:

In [8]:
data['date'] = data['event_timestamp'].dt.date

In [9]:
data['date'] = pd.to_datetime(data['date'])

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243713 entries, 0 to 244125
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   event_name       243713 non-null  object        
 1   id               243713 non-null  int64         
 2   event_timestamp  243713 non-null  datetime64[ns]
 3   exp_id           243713 non-null  int64         
 4   date             243713 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 11.2+ MB


### Изучим и проверим данные

#### Количество событий 

Заглянем в столбец событий:

In [11]:
total_events = data.event_name.count()
print('Всего событий в логе:', total_events)

Всего событий в логе: 243713


In [12]:
data.event_name.value_counts()

MainScreenAppear           119101
OffersScreenAppear          46808
CartScreenAppear            42668
PaymentScreenSuccessful     34118
Tutorial                     1018
Name: event_name, dtype: int64

Численно события соответствуют воронке продаж - от "главного экрана" до "оплаты" количество событий убывает.

#### Количество пользователей 

In [13]:
all_users = data.id.nunique()

In [14]:
print('Всего уникальных пользователей в логе:', all_users)

Всего уникальных пользователей в логе: 7551


#### Сколько в среднем событий приходится на пользователя

In [17]:
print('В среднем на одного пользователя приходится', int(data.groupby('id')['event_name'].count().mean().round()), 'события.')

В среднем на одного пользователя приходится 32 события.


Посмотрим также медиану:

In [18]:
print('В среднем по медиане на одного пользователя приходится', int(data.groupby('id')['event_name'].count().median().round()), 'событий.')

AttributeError: 'float' object has no attribute 'round'

Значит есть несколько пользователей с очень большим количеством событий, которые смещают арифметическое среднее в большую сторону.

#### Данные о дате и их корректировка

In [ ]:
print('Минимальная дата', data.event_timestamp.min())

In [ ]:
print('Максимальная дата', data.event_timestamp.max())

In [ ]:
data['event_timestamp'].hist(bins=50, figsize=(18, 7));

Видно, что данные распределены неравномерно - до 2019-08-01 данных совсем мало, отбросим их. 

In [ ]:
data = data.query('date >= "2019-08-01"')

In [ ]:
data['event_timestamp'].hist(bins=8, figsize=(14, 7));

Теперь данные для исследования распределены равномерно. 

In [ ]:
print('Минимальная дата', data.event_timestamp.min())

In [ ]:
print('Максимальная дата', data.event_timestamp.max())

Рассматриваемый период - 7 дней.

#### Очищение данных от "событий из прошлого"

Проверим, не попали ли в выбранный период события из прошлого - данные пользователей, которые посещали страницу до выбранного периода 2019-08-01. Для этого оставим только тех пользователей, действия которых начинаются с посещения главной страницы - **event_name = MainScreenAppear.**

Создадим список id нужных пользователей, которые не были знакомы с приложением до даты эксперимента:

In [ ]:
clean_users = data.loc[data['event_name'] == 'MainScreenAppear', 'id'].unique()
len(clean_users)

Удалим из датафрейма id пользователей, которые не входят в этот список:

In [ ]:
data = data[data['id'].isin(clean_users)]

Обновим количество событий после корректировки даты и пользователей:

In [ ]:
clean_events = data.shape[0]
clean_events

In [ ]:
print('Мы удалили не более', round(100 - ((len(clean_users) / all_users) * 100), 1), '% пользователей.')

In [ ]:
print('Мы удалили не более', round(100 - ((clean_events / total_events) * 100), 1), '% событий.')

Проверим также повторяются ли пользователи в других группах:

In [ ]:
data.groupby('id').agg({'exp_id' : 'nunique'}).query('exp_id > 1')

Пользователи во всех группах уникальные.

In [ ]:
# Код ревьюера
# Проверим пользователей, которые могли участвовать в двух или нескольких группах одновременно:
data.groupby('id').agg({'exp_id':'nunique'}).query('exp_id > 1') 

#### Данные о пользователях по трём группам

Проверим, что у нас есть пользователи из всех трёх экспериментальных групп:

In [ ]:
data.groupby('exp_id', as_index=False)['id'].nunique()

У нас есть примерно одинаковое количество пользователей в трёх группах.

In [ ]:
data['exp_id'].value_counts()

Количество событий в группах также примерно одинаковое.

### Воронка событий

In [ ]:
data.event_name.value_counts()

Изучим воронку событий: наиболее частое событие - просмотр главного экрана **MainScreenAppear**, это логично, ведь это самое первое действие пользователя. Далее пользователь знакомится с предложением **OffersScreenAppear**, добавляет его в корзину **CartScreenAppear** и оплачивает **PaymentScreenSuccessful**. Руководство по пользованию приложением **Tutorial** нужно далеко не всем, пользователь может заглянуть в него в любое время или не посмотреть вообще.  

Исключим событие **Tutorial** в дальнейших исследованиях.

In [ ]:
data = data.query('event_name != "Tutorial"')

Посчитаем, сколько пользователей совершали каждое событие и их долю.  
А также, какая доля пользователей проходит на следующий шаг воронки по отношению к предыдущему шагу:

In [ ]:
all_events = data.groupby('event_name', as_index=False).agg({'id' : 'nunique'}).sort_values(by='id', ascending=False)
all_events['share_%'] = (all_events['id'] / all_events.loc[1, 'id']).round(2) * 100
all_events['share_next_%'] = (((all_events['id'] / all_events['id'].shift(1)).round(2) * 100) - 100).abs()
all_events

In [ ]:
x = list(all_events['id'])
y = list(all_events['event_name'])
fig = px.funnel(all_events, x=x, y=y)
fig.update_layout(title="Воронка конверсии (по пользователям)")
fig.show()

Больше всего пользователей теряется на втором шаге - переход с главного экрана **MainScreenAppear** на экран предложения **OffersScreenAppear**.

**48%** пользователей доходит от первого события до оплаты.

### Результаты A/A/B - эксперимента

Количество пользователей в каждой группе:

In [37]:
data.groupby('exp_id', as_index=False)['id'].nunique()

,exp_id,id
0,246,2450
1,247,2476
2,248,2493


Разделим данные по трём группам:

In [38]:
dt_246 = data.query('exp_id == 246')
dt_247 = data.query('exp_id == 247')
dt_248 = data.query('exp_id == 248')

Посмотрим на соотношение событий в каждой группе:

In [39]:
d_246 = dt_246.groupby('event_name', as_index=False).agg({'id' : 'nunique'}).sort_values(by='id', ascending=False)
d_246.reset_index(drop= True , inplace= True )
d_246

,event_name,id
0,MainScreenAppear,2450
1,OffersScreenAppear,1509
2,CartScreenAppear,1236
3,PaymentScreenSuccessful,1170


In [40]:
d_247 = dt_247.groupby('event_name', as_index=False).agg({'id' : 'nunique'}).sort_values(by='id', ascending=False)
d_247.reset_index(drop= True , inplace= True )
d_247

,event_name,id
0,MainScreenAppear,2476
1,OffersScreenAppear,1484
2,CartScreenAppear,1207
3,PaymentScreenSuccessful,1128


In [41]:
event = data['event_name'].unique()
event

array(['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear',
       'PaymentScreenSuccessful'], dtype=object)

#### A/A-эксперимент - проверка статистической значимости

Проверим статистические различия между группами А и А. Используем z-тест для определения различий в долях.

Создадим функцию для быстрого проведения z-теста по всем группам:

In [42]:
def z_test(a1, a2, event, alpha, n):
    # критический уровень статистической значимости
    alpha = alpha / n 
    # уникальные пользователи совершившие определенное событие в группе А1 и А2
    success = np.array([data.query('event_name == @event and exp_id == @a1').id.nunique(), 
                        data.query('event_name == @event and exp_id == @a2').id.nunique()])
    # всего число уникальных пользователей в группе А1 и А2
    users = np.array([data.query('exp_id == @a1').id.nunique(),
                      data.query('exp_id == @a2').id.nunique()])

    # пропорция успехов в группах:
    p1 = success[0]/users[0]
    p2 = success[1]/users[1]

    # пропорция успехов в комбинированном датасете:
    p_combined = (success[0] + success[1]) / (users[0] + users[1])

    # разница пропорций в датасетах
    difference = p1 - p2
    
    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/users[0] + 1/users[1]))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1) 
    
    # двусторонний тест
    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('\033[1m' + 'Событие:', event + '\033[0m')
    print('p-значение:', (p_value).round(3))

    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

In [43]:
for event in ['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful']:
    z_test(246, 247, event, 0.05, 16)
    print()

Событие: MainScreenAppear
p-значение: nan
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: OffersScreenAppear
p-значение: 0.234
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: CartScreenAppear
p-значение: 0.233
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: PaymentScreenSuccessful
p-значение: 0.122
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными



Комментарий: поскольку мы очистили данные от старых пользователей, мы имеем 100% посещаемость главной страницы. Поэтому разницы в первом событии здесь и далее не будет.

Контрольные группы А и А не имеют статистически значимых различий между долями, а значит механизм распределения данных по группам работает корректно и мы можем уверенно опираться на эти данные в дальнейшем сравнении с группой В.

#### Самое популярное событие, число и доля пользователей.

In [44]:
event_246 = dt_246.groupby('event_name', as_index=False).agg({'id' : 'count'}).sort_values(by='id', ascending=False)
event_246.reset_index(drop= True , inplace= True )
event_246.columns = ['event_name', 'event_246']
event_246

,event_name,event_246
0,MainScreenAppear,37676
1,OffersScreenAppear,14488
2,CartScreenAppear,14413
3,PaymentScreenSuccessful,11603


In [45]:
event_247 = dt_247.groupby('event_name', as_index=False).agg({'id' : 'count'}).sort_values(by='id', ascending=False)
event_247.reset_index(drop= True , inplace= True )
event_247.columns = ['event_name', 'event_247']
event_247

,event_name,event_247
0,MainScreenAppear,39090
1,OffersScreenAppear,14845
2,CartScreenAppear,12132
3,PaymentScreenSuccessful,9746


Самое популярное событие в обеих группах - это просмотр главного экрана **MainScreenAppear**.

Добавим в таблицы количество уникальных пользователей, совершивших каждое действие и их долю от общего числа:

In [46]:
users_246 = dt_246.groupby('event_name', as_index=False).agg({'id' : 'nunique'}).sort_values(by='id', ascending=False)
users_246.reset_index(drop= True , inplace= True )
event_246['uniq_users'] = users_246['id']
event_246['share_%'] = (event_246['uniq_users'] / event_246.loc[0, 'uniq_users']).round(2) * 100
event_246

,event_name,event_246,uniq_users,share_%
0,MainScreenAppear,37676,2450,100.0
1,OffersScreenAppear,14488,1509,62.0
2,CartScreenAppear,14413,1236,50.0
3,PaymentScreenSuccessful,11603,1170,48.0


In [47]:
users_247 = dt_247.groupby('event_name', as_index=False).agg({'id' : 'nunique'}).sort_values(by='id', ascending=False)
users_247.reset_index(drop= True , inplace= True )
event_247['uniq_users'] = users_247['id']
event_247['share_%'] = (event_247['uniq_users'] / event_247.loc[0, 'uniq_users']).round(2) * 100
event_247

,event_name,event_247,uniq_users,share_%
0,MainScreenAppear,39090,2476,100.0
1,OffersScreenAppear,14845,1484,60.0
2,CartScreenAppear,12132,1207,49.0
3,PaymentScreenSuccessful,9746,1128,46.0


#### A/В - эксперимент

 Проверим с помощью z-теста есть ли статистически значимые различия между контрольными группами А/А и экспериментальной В:

In [48]:
for event in ['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful']:
    z_test(246, 248, event, 0.05, 16)
    print()

Событие: MainScreenAppear
p-значение: nan
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: OffersScreenAppear
p-значение: 0.18
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: CartScreenAppear
p-значение: 0.064
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: PaymentScreenSuccessful
p-значение: 0.179
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными



Статистически значимой разницы между группами А (246) и В нет.

In [49]:
for event in ['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful']:
    z_test(247, 248, event, 0.05, 16)
    print()

Событие: MainScreenAppear
p-значение: nan
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: OffersScreenAppear
p-значение: 0.881
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: CartScreenAppear
p-значение: 0.51
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: PaymentScreenSuccessful
p-значение: 0.837
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными



Статистически значимой разницы между группами А (247) и В нет.

Проверим также различия между обеими контрольными группами А+А и группой В. Объединим контрольные группы в один датафрейм:

In [50]:
dt_249 = data.query('exp_id == 246 or exp_id == 247')
dt_249['exp_id'] = dt_249['exp_id'].map(lambda x: '249'.format(x)).astype(int)
dt_249.head()

,event_name,id,event_timestamp,exp_id,date
2829,MainScreenAppear,3737462046622621720,2019-08-01 00:08:00,249,2019-08-01
2830,MainScreenAppear,3737462046622621720,2019-08-01 00:08:55,249,2019-08-01
2831,OffersScreenAppear,3737462046622621720,2019-08-01 00:08:58,249,2019-08-01
2832,MainScreenAppear,1433840883824088890,2019-08-01 00:08:59,249,2019-08-01
2833,MainScreenAppear,4899590676214355127,2019-08-01 00:10:15,249,2019-08-01


In [51]:
dt_249.id.nunique()

4926

In [52]:
users_per_group = data.pivot_table(index='event_name', columns='exp_id', values='id', aggfunc='nunique')
users_per_group = users_per_group.sort_values(by=246, ascending=False).reset_index()
users_per_group

exp_id,event_name,246,247,248
0,MainScreenAppear,2450,2476,2493
1,OffersScreenAppear,1509,1484,1489
2,CartScreenAppear,1236,1207,1192
3,PaymentScreenSuccessful,1170,1128,1143


In [53]:
# Проведем АAВ тест, для этого подготовим данные - подсчитаем общее количество пользователей:
#users_per_group_control = users_per_group.copy()
#users_per_group_control.loc[247] += users_per_group_control.loc[246]
#users_per_group_control.drop(246, inplace=True)
#users_per_group_control

In [54]:
def z_test_aa(a1, a2, event, alpha, n):
    # критический уровень статистической значимости
    alpha = alpha / n 
    # уникальные пользователи совершившие определенное событие в группе А1 и А2
    success = np.array([dt_249.query('event_name == @event and exp_id == @a1').id.nunique(), 
                      data.query('event_name == @event and exp_id == @a2').id.nunique()])
    # всего число уникальных пользователей в группе А1 и А2
    users = np.array([dt_249.query('exp_id == @a1').id.nunique(),
                      data.query('exp_id == @a2').id.nunique()])

    # пропорция успехов в группах:
    p1 = success[0]/users[0]
    p2 = success[1]/users[1]

    # пропорция успехов в комбинированном датасете:
    p_combined = (success[0] + success[1]) / (users[0] + users[1])

    # разница пропорций в датасетах
    difference = p1 - p2
    
    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / np.sqrt(p_combined * (1 - p_combined) * (1/users[0] + 1/users[1]))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1) 
    
    # двусторонний тест
    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('\033[1m' + 'Событие:', event + '\033[0m')
    print('p-значение:', (p_value).round(3))

    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

In [55]:
for event in ['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful']:
    z_test_aa(249, 248, event, 0.05, 16)
    print()

Событие: MainScreenAppear
p-значение: nan
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: OffersScreenAppear
p-значение: 0.391
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: CartScreenAppear
p-значение: 0.147
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: PaymentScreenSuccessful
p-значение: 0.513
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными



Статистически значимой разницы между объединенной контрольной группой А (246 + 247) и В нет.

Исходя из всех проведенных исследований мы с уверенностью можем сказать, что нет статистичеки значимого влияния новых шрифтов на приложение - новые шрифты не лучше и не хуже старых.

#### уровень значимости

- Для проверки статистических гипотез был выбран уровень статистической значимости 0.05 и была применена поправка Бонферрони
- Всего было сделано 16 проверок статистических гипотез